In [2]:
import os
import openai

from pathlib import Path
from dotenv import load_dotenv

from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema.messages import HumanMessage, AIMessage, SystemMessage

from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)

load_dotenv()
openai.organization = os.getenv("OPENAI_ORG")
openai.api_key = os.getenv("OPENAI_API_KEY")

model = "gpt-3.5-turbo-1106"
temperature = 0.1

In [3]:
llm = ChatOpenAI(
    model_name=model,
    temperature=temperature,
)

prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template("This is the character"),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{message}"),
    ]
)

memory = ConversationBufferMemory(
    memory_key="chat_history", return_messages=True
)

LLMChain(llm=llm, prompt=prompt, verbose=False, memory=memory)

LLMChain(memory=ConversationBufferMemory(return_messages=True, memory_key='chat_history'), prompt=ChatPromptTemplate(input_variables=['chat_history', 'message'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='This is the character')), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['message'], template='{message}'))]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001D0C6B211B0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001D0D8370B20>, model_name='gpt-3.5-turbo-1106', temperature=0.1, op